In [46]:
import geopandas as gpd
from fiona import crs

def calculate_adjacent_elevation_average(input_gpkg, output_gpkg):
    # Load the geopackage
    gdf = gpd.read_file(input_gpkg)
    
    # Ensure the geometry operations use pygeos backend, if available
    gpd.params.use_pygeos = True
    
    # Create a 1m buffer around each polygon
    gdf['buffer'] = gdf.geometry.buffer(1)
    
    # Perform a spatial join
    joined_gdf = gpd.sjoin(gdf[['geometry', 'Elevation Mean', 'buffer']], gdf, how='left', op='intersects')
    
    # Exclude self in the adjacency calculation
    joined_gdf = joined_gdf[joined_gdf.index != joined_gdf.index_right]
    
    # Calculate the mean elevation of adjacent polygons
    elevation_avg = joined_gdf.groupby(joined_gdf.index)['Elevation Mean_right'].mean()
    
    # Merge the average elevation back to the original GeoDataFrame
    gdf = gdf.join(elevation_avg.rename('Ele0vation Average'), how='left')
    
    # Set geometry column explicitly
    gdf.set_geometry('geometry', inplace=True)
    #drop the buffer column
    gdf.drop(columns='buffer', inplace=True)
    # Define a schema to explicitly declare types for each column


    # Write the updated GeoDataFrame to a new geopackage
    gdf.to_file(output_gpkg, driver='GPKG')

In [47]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
import pandas as pd

def calculate_distance(gdf):
    gdf['Centroid'] = gdf['geometry'].centroid
    gdf['buffer'] = gdf.geometry.buffer(1)  # Ensures adjacency
    joined_gdf = gpd.sjoin(gdf[['geometry', 'Centroid', 'Elevation Mean', 'Slope Mean']], 
                           gdf[['geometry', 'Centroid', 'Elevation Mean', 'Slope Mean']], 
                           how='left', 
                           predicate='intersects')
    joined_gdf = joined_gdf[joined_gdf.index != joined_gdf.index_right]
    joined_gdf['Distance'] = joined_gdf.apply(lambda row: row['Centroid_left'].distance(row['Centroid_right']), axis=1)
    return joined_gdf

def get_adjacent_slopes(input_gpkg, output_gpkg):
    gdf = gpd.read_file(input_gpkg)
    joined_gdf = calculate_distance(gdf)

    # Assign directional distances and slopes
    joined_gdf['Distance Upstream'] = np.where(joined_gdf['Elevation Mean_left'] < joined_gdf['Elevation Mean_right'], joined_gdf['Distance'], np.nan)
    joined_gdf['Distance Downstream'] = np.where(joined_gdf['Elevation Mean_left'] > joined_gdf['Elevation Mean_right'], joined_gdf['Distance'], np.nan)
    joined_gdf['Slope Upstream'] = np.where(joined_gdf['Elevation Mean_left'] < joined_gdf['Elevation Mean_right'], joined_gdf['Slope Mean_right'], np.nan)
    joined_gdf['Slope Downstream'] = np.where(joined_gdf['Elevation Mean_left'] > joined_gdf['Elevation Mean_right'], joined_gdf['Slope Mean_right'], np.nan)

    # Calculate the mean of the values grouped by the original index
    final_stats = joined_gdf.groupby(joined_gdf.index).agg({
        'Distance Upstream': 'mean',
        'Distance Downstream': 'mean',
        'Slope Upstream': 'mean',
        'Slope Downstream': 'mean'
    })

    # Merge the results back to the original GeoDataFrame
    gdf = gdf.join(final_stats, how='left')

    # Set geometry column explicitly and clean up data
    gdf.set_geometry('geometry', inplace=True)
    gdf.drop(columns=['buffer', 'Centroid'], inplace=True)

    # Write the updated GeoDataFrame to a new geopackage
    gdf.to_file(output_gpkg, driver='GPKG')

    return gdf

def calculate_central_slope_difference(input_gpkg, output_gpkg):
    gdf = gpd.read_file(input_gpkg)
    #check if slope upstream exists
    if 'Slope Upstream' not in gdf.columns:
        gdf = get_adjacent_slopes(input_gpkg, output_gpkg)

    # Define a function to calculate central difference using dynamic h
    def central_diff(row):
        if np.isnan(row['Distance Upstream']) or np.isnan(row['Distance Downstream']):
            return None
        return (row['Slope Downstream'] - row['Slope Upstream']) / (row['Distance Upstream'] + row['Distance Downstream'])

    # Apply function to calculate central slope differences
    gdf['Central Slope Difference'] = gdf.apply(central_diff, axis=1)

    # Write the updated GeoDataFrame to a new geopackage
    gdf.to_file(output_gpkg, driver='GPKG')

    return gdf

def calculate_backward_slope_difference(input_gpkg, output_gpkg):
    gdf = gpd.read_file(input_gpkg)
    #check if slope upstream exists
    if 'Slope Upstream' not in gdf.columns:
        gdf = get_adjacent_slopes(input_gpkg, output_gpkg)

    # Define a function to calculate backward difference direction based on elevation
    def backward_diff(row):
        if np.isnan(row['Distance Upstream']):
            return None
        return (row['Slope Mean'] - row['Slope Upstream']) / (row['Distance Upstream'])

    # Apply function to calculate backward slope differences
    gdf['Backward Slope Difference'] = gdf.apply(backward_diff, axis=1)

    # Write the updated GeoDataFrame to a new geopackage
    gdf.to_file(output_gpkg, driver='GPKG')

    return gdf

def calculate_forward_slope_difference(input_gpkg, output_gpkg):
    gdf = gpd.read_file(input_gpkg)
    #check if slope upstream exists
    if 'Slope Upstream' not in gdf.columns:
        gdf = get_adjacent_slopes(input_gpkg, output_gpkg)

    # Define a function to calculate forward difference direction based on elevation
    def forward_diff(row):
        if np.isnan(row['Distance Downstream']):
            return None
        return (row['Slope Downstream'] - row['Slope Mean']) / (row['Distance Downstream'])

    # Apply function to calculate forward slope differences
    gdf['Forward Slope Difference'] = gdf.apply(forward_diff, axis=1)

    # Write the updated GeoDataFrame to a new geopackage
    gdf.to_file(output_gpkg, driver='GPKG')

    return gdf


# Curvature Calculation for Digital Curves

The curvature of a curve at any given point measures how quickly the curve deviates from a straight trajectory, which can be thought of as how "curved" the curve is at that point. For digital curves, which consist of discrete points rather than a continuous function, we need to adapt traditional curvature calculation methods to fit this discrete nature.

## Methodology

The method used here approximates curvature by measuring the angular change between consecutive segments along the curve. Each segment is defined by three consecutive points, and the curvature is estimated based on the angle formed at the middle point.

### Steps of Curvature Calculation:

1. **Vector Calculation**:
   For three consecutive points $p_1, p_2, p_3$, compute two vectors:
   - $\vec{v1} = p_2 - p_1$
   - $\vec{v2} = p_3 - p_2$

2. **Angle Between Vectors**:
   The angle $\theta$ between $\vec{v1}$ and $\vec{v2}$ is calculated using the dot product formula:
   $$
   \cos(\theta) = \frac{\vec{v1} \cdot \vec{v2}}{\|\vec{v1}\| \|\vec{v2}\|}
   $$
   And then,
   $$
   \theta = \arccos(\cos(\theta))
   $$
   where $\arccos$ is the arc cosine function, which returns the angle in radians for a given cosine value.

3. **Curvature Estimation**:
   The curvature $\kappa$ at $p_2$ is then estimated as:
   $$
   \kappa = \frac{\theta}{\|\vec{v1} + \vec{v2}\|}
   $$
   This represents the angular change per unit length, approximating the curvature at $p_2$.

4. **Maximum Curvature**:
   The function iterates over all possible consecutive point triplets in the line and calculates the curvature for each. It then returns the maximum of these curvature values as the overall curvature of the curve.

## Usage

The function `calculate_curvature(line)` can be called with a `LineString` object containing the coordinates of the curve points. It returns the maximum curvature found among all evaluated points, which identifies the point of highest curvature along the digital curve.

This method is particularly useful for digital geometry applications where curves are represented as sequences of discrete points rather than continuous functions.

Source: 

https://researchspace.auckland.ac.nz/bitstream/handle/2292/2787/CITR-TR-183.pdf?sequence=1&isAllowed=y


In [48]:
import geopandas as gpd
from shapely.geometry import LineString
import numpy as np

def calculate_curvature(line):
    """
    Calculate the curvature of a line using an angular measure method suitable for discrete data points.
    This approach is inspired by methods suitable for digital geometry, as discussed in the paper.
    """
    if len(line.coords) < 3:
        return 0  # Not enough points to calculate curvature

    coords = np.array(line.coords)
    curvatures = []

    for i in range(1, len(coords) - 1):
        p1 = coords[i - 1]
        p2 = coords[i]
        p3 = coords[i + 1]

        # Vector from p1 to p2
        v1 = p2 - p1
        # Vector from p2 to p3
        v2 = p3 - p2

        # Calculate the angle between v1 and v2 using the dot product and norm
        angle_cos = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
        angle_cos = np.clip(angle_cos, -1.0, 1.0)  # Clip the cos value to the valid range of acos
        angle = np.arccos(angle_cos)

        # Estimate curvature as the angle change per unit length
        curvature = angle / np.linalg.norm(v1 + v2)
        curvatures.append(curvature)

    # Return the maximum curvature estimated
    return max(curvatures) if curvatures else 0

def add_curvature_field(input_gpkg_path, centerline_gpkg_path, output_gpkg_path):
    # Load geopackage
    gdf = gpd.read_file(input_gpkg_path)
    centerline = gpd.read_file(centerline_gpkg_path)

    # Prepare a column for curvature
    gdf['Curvature'] = 0

    # Assuming there is only one centerline geometry
    centerline_geometry = centerline.geometry.squeeze()

    for index, polygon in gdf.iterrows():
        # Calculate the intersection of the centerline with the current polygon
        intersection = centerline_geometry.intersection(polygon.geometry)
        if intersection.is_empty:
            print(f"Polygon {index} does not intersect with the centerline")
        if not intersection.is_empty:
            # Ensure the intersection is a linestring
            if isinstance(intersection, LineString):
                curvature = calculate_curvature(intersection)
            else:
                # Handle cases where intersection might be MultiLineString or other geometry types
                curvature = max(calculate_curvature(line) for line in intersection if isinstance(line, LineString))
            gdf.at[index, 'Curvature'] = curvature

    # Save to a new geopackage
    gdf.to_file(output_gpkg_path, driver="GPKG")


In [49]:
def get_adjacent_fields(gdf, field_name_list):
    field_name_list = field_name_list if 'geometry' in field_name_list else field_name_list + ['geometry']
    field_name_list = field_name_list if 'Elevation Mean' in field_name_list else field_name_list + ['Elevation Mean']
    gdf['buffer'] = gdf.geometry.buffer(0.1)  # Ensures adjacency

    joined_gdf = gpd.sjoin(gdf[field_name_list], gdf[field_name_list], how='left', predicate='intersects') 
    joined_gdf = joined_gdf[joined_gdf.index != joined_gdf.index_right]
    return joined_gdf

def add_adjacent_fields(input_gpkg, output_gpkg, field_list):
    gdf = gpd.read_file(input_gpkg)
    joined_gdf = get_adjacent_fields(gdf, field_list)
    
    for field in field_list:
        joined_gdf[f'{field} Upstream'] = np.where(joined_gdf['Elevation Mean_left'] < joined_gdf['Elevation Mean_right'], joined_gdf[field + '_right'], np.nan)
        joined_gdf[f'{field} Downstream'] = np.where(joined_gdf['Elevation Mean_left'] > joined_gdf['Elevation Mean_right'], joined_gdf[field + '_right'], np.nan)
        # Assign directional distances and slopes

    groupby_dict = {f'{field} Upstream': 'mean' for field in field_list}
    groupby_dict.update({f'{field} Downstream': 'mean' for field in field_list})
    # Calculate the mean of the values grouped by the original index
    final_stats = joined_gdf.groupby(joined_gdf.index).agg({ **groupby_dict })

    # Merge the results back to the original GeoDataFrame
    gdf = gdf.join(final_stats, how='left')

    # Set geometry column explicitly and clean up data
    gdf.set_geometry('geometry', inplace=True)
    gdf.drop(columns=['buffer'], inplace=True)

    # Write the updated GeoDataFrame to a new geopackage
    gdf.to_file(output_gpkg, driver='GPKG')

    return gdf

def calculate_central_difference(input_gpkg, output_gpkg, field_name, drop_adj_fields = True):
    gdf = gpd.read_file(input_gpkg)
    #check if slope upstream exists
    if f'{field_name} Upstream' not in gdf.columns or f'{field_name} Downstream' not in gdf.columns:
        print(len([field_name]))
        print([field_name])
        gdf = add_adjacent_fields(input_gpkg, output_gpkg, [field_name])

    # Define a function to calculate central difference using dynamic h
    def central_diff(row):
        if np.isnan(row['Distance Upstream']) or np.isnan(row['Distance Downstream']):
            return 0
        return (row[f'{field_name} Downstream'] - row[f'{field_name} Upstream']) / (row['Distance Upstream'] + row['Distance Downstream'])

    # Apply function to calculate central slope differences
    gdf[f'{field_name} Central Diff'] = gdf.apply(central_diff, axis=1)
    if drop_adj_fields:
        gdf.drop(columns=[f'{field_name} Upstream', f'{field_name} Downstream'], inplace=True)
    # Write the updated GeoDataFrame to a new geopackage
    gdf.to_file(output_gpkg, driver='GPKG')

    return gdf



In [50]:
from segment_stream import create_smooth_perpendicular_lines

def calculate_width(input_gpkg, centerline_gpkg, output_gpkg):
    # Load the geopackage and extract layers
    gdf = gpd.read_file(input_gpkg)
    perpendiculars_gdf = create_smooth_perpendicular_lines(centerline_gpkg, line_length=500, spacing=1, window=20)

    # Ensure the CRS of the perpendiculars matches the CRS of the input gdf
    if perpendiculars_gdf.crs != gdf.crs:
        perpendiculars_gdf = perpendiculars_gdf.to_crs(gdf.crs)

    def clip_and_average_lengths(polygon, perpendiculars):
        # Clip perpendiculars to the polygon
        clipped_perpendiculars = perpendiculars.intersection(polygon)
        
        # Get catesian lengths of the clipped perpendiculars
        lengths = [perpendicular.length for perpendicular in clipped_perpendiculars]
        # Return the average length of clipped perpendiculars
        #get rid of the zero lengths
        lengths = [length for length in lengths if length > 0]
        return np.mean(lengths) if lengths else 0

    # Calculate average length of clipped perpendiculars for each polygon
    gdf['Width'] = gdf.geometry.apply(lambda polygon: clip_and_average_lengths(polygon, perpendiculars_gdf))

    # Write the results to a new GeoPackage
    gdf.to_file(output_gpkg, driver='GPKG')

    return gdf

def add_slope_over_width(input_gpkg):
    gdf = gpd.read_file(input_gpkg)
    gdf['Slope Over Width'] = gdf['Slope Mean'] / gdf['Width']
    gdf.to_file(input_gpkg, driver='GPKG')

    return gdf

def add_change_in_slope_over_width(input_gpkg):
    gdf = gpd.read_file(input_gpkg)
    gdf['Change in Slope Over Width'] = gdf['Central Slope Difference'] / gdf['Width']
    gdf.to_file(input_gpkg, driver='GPKG')

    return gdf

def add_change_in_slope_over_area(input_gpkg):
    gdf = gpd.read_file(input_gpkg)
    gdf['Change in Slope Over Area'] = gdf['Central Slope Difference'] / gdf['Area']
    gdf.to_file(input_gpkg, driver='GPKG')

    return gdf

def add_slope_over_area(input_gpkg):
    gdf = gpd.read_file(input_gpkg)
    gdf['Slope Over Area'] = gdf['Slope Mean'] / gdf['Area']
    gdf.to_file(input_gpkg, driver='GPKG')

    return gdf


def calc_net_change(input_gpkg):
    gdf = gpd.read_file(input_gpkg)
    
    # Replace NaN and None with zero for calculations
    columns_to_clean = ['Erosion Sum', 'Deposition Sum', 'Erosion Count', 'Deposition Count']
    for column in columns_to_clean:
        gdf[column] = pd.to_numeric(gdf[column], errors='coerce').fillna(0)

    # Compute net change per unit area
    erosion_sum = gdf['Erosion Sum'].values
    deposition_sum = gdf['Deposition Sum'].values
    erosion_count = gdf['Erosion Count'].values
    deposition_count = gdf['Deposition Count'].values
    
    gdf['Net Change'] = deposition_sum + erosion_sum

    # Compute normalized net change, accounting for division by zero
    total_count = deposition_count + erosion_count
    with np.errstate(divide='ignore', invalid='ignore'):
        gdf['Net Change Norm'] = np.where(total_count == 0, 0, (deposition_sum + erosion_sum) / total_count)
    
    # Save changes back to the GeoPackage
    gdf.to_file(input_gpkg, driver='GPKG')

    return gdf


def calc_net_sfm_change(input_gpkg):
    gdf = gpd.read_file(input_gpkg)
    
    # Replace NaN and None with zero for calculations
    columns_to_clean = ['Sfm Erosion Sum', 'Sfm Deposition Sum', 'Sfm Erosion Count', 'Sfm Deposition Count']
    for column in columns_to_clean:
        gdf[column] = pd.to_numeric(gdf[column], errors='coerce').fillna(0)

    # Compute net change per unit area
    erosion_sum = gdf['Sfm Erosion Sum'].values
    deposition_sum = gdf['Sfm Deposition Sum'].values
    erosion_count = gdf['Sfm Erosion Count'].values
    deposition_count = gdf['Sfm Deposition Count'].values
    
    gdf['Sfm Net Change'] = deposition_sum - erosion_sum

    # Compute normalized net change accounting for division by zero
    total_count = deposition_count + erosion_count
    with np.errstate(divide='ignore', invalid='ignore'):
        gdf['Sfm Net Change Norm'] = np.where(total_count == 0, 0, (deposition_sum - erosion_sum) / total_count)
    
    # Save changes back to the GeoPackage
    gdf.to_file(input_gpkg, driver='GPKG')

    return gdf



In [51]:
import numpy as np
import rasterio
from rasterio.mask import mask
import os

def aggregate_raster_stats(gdf, output_path, params):
    """Aggregate statistics from a raster based on a GeoDataFrame shape.

    Args:
        raster_path (str): Path to the raster file.
        gdf (GeoDataFrame): GeoDataFrame containing the shapes.
        output_path (str): Path to save the modified GDF.
        params (dict): Dictionary containing params for aggregation including:
                        - raster_path (str): Path to the raster file.
                        - threshold (float, optional): Value to filter the raster data above or below.
                        - threshold_direction (str): 'above' or 'below', direction for threshold filtering.
                        - raster_value_to_match: Value(s) that raster cells must match to be included.
                        - stat_key (dict): Dictionary mapping statistical operation to field name in GDF.

    Returns:
        GeoDataFrame: The updated GeoDataFrame.
    """
    valid_stat_list = ['mean', 'median', 'max', 'min', 'std', 'sum', 'percent', 'count', 'norm_sum']
    #specify valid percentile values
    valid_stat_list.extend([f'Q_{i}' for i in range(1, 100)])
    # Make all words in stat_key lowercase
    params['stat_key'] = {key.lower(): value for key, value in params['stat_key'].items()}
    if not all(stat in valid_stat_list for stat in params['stat_key']):
        print(f"User Specified statistics in stat_key:\n {params['stat_key']}")
        print(f"\nValid statistical operations are: {valid_stat_list}")
        raise ValueError("Invalid statistical operation specified in stat_key")
    
    raster_path = params.get('raster_path', None)
    if raster_path is None:
        print("\nParams provided do not contain raster_path")
        for key, value in params.items():
            print(key, value)
        raise ValueError("raster_path is required in params")
    with rasterio.open(raster_path) as src:
        no_data = src.nodata

        # Extract params
        threshold = params.get('threshold', None)
        threshold_direction = params.get('threshold_direction', 'above')
        raster_value_to_match = params.get('raster_value_to_match', None)
        stat_key = params.get('stat_key', {})
        
        # Prepare the GDF by adding new columns for specified stats
        for stat in stat_key:
            gdf[stat_key[stat]] = np.nan

        # Process each geometry
        for index, row in gdf.iterrows():
            shapes = gdf.iloc[[index]]
            out_image, out_transform = mask(src, shapes.geometry, crop=True, all_touched=True)
            data = out_image[out_image != no_data]

            # Filter data by raster_value_to_match if specified
            if raster_value_to_match is not None:
                data = np.where(np.isin(data, raster_value_to_match), data, np.nan)

            # Apply threshold filtering if specified
            if threshold is not None:
                if threshold_direction == 'above':
                    data = data[data > threshold]
                elif threshold_direction == 'below':
                    data = data[data < threshold]

            # Calculate statistics and update GDF
            for stat in stat_key:
                if data.size > 0:
                    if stat == 'mean':
                        result = np.mean(data)
                    elif stat == 'median':
                        result = np.median(data)
                    elif stat == 'max':
                        result = np.max(data)
                    elif stat == 'min':
                        result = np.min(data)
                    #the string 'Q_xx' is used to represent the xxth percentile
                    elif stat[0] == 'Q':
                        percentile = int(stat.split('_')[1])
                        result = np.percentile(data, percentile)
                    elif stat == 'std':
                        result = np.std(data)
                    elif stat == 'sum':
                        result = np.sum(data)
                    elif stat == 'norm_sum':
                        result = np.sum(data) / data.size
                    elif stat == 'percent':
                        result = np.count_nonzero(data) / data.size * 100
                    elif stat == 'count':
                        result = np.count_nonzero(data)
                else:
                    result = np.nan
                gdf.at[index, stat_key[stat]] = result

        # Output the modified shapefile if a path is provided
        if output_path is not None:
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            gdf.to_file(output_path)

        return gdf


In [52]:
def add_area_field(gdf, output_shapefile_path=None):

    if type(gdf) == str:
        gdf = gpd.read_file(gdf)
        
    # Compute area and perimeter for each sub-shape
    gdf['Area'] = gdf['geometry'].area
    #gdf['Perimeter'] = gdf['geometry'].length
    
    if output_shapefile_path is not None:
        gdf.to_file(output_shapefile_path)
        
    return gdf

In [53]:
def clean_gdf(gdf, watershed_col='Watershed'):
    """
    Cleans a GeoDataFrame by replacing invalid numeric values with 0 and ensuring all columns,
    except the specified 'watershed' column, are of type string.
    
    Parameters:
    gdf (GeoDataFrame): The GeoDataFrame to clean.
    watershed_col (str): The column name for the watershed, which will not be converted to string.
    
    Returns:
    GeoDataFrame: The cleaned GeoDataFrame.
    """
    # Replace invalid numeric values with 0 for all columns except the watershed column
    numeric_cols = gdf.select_dtypes(include=[np.number]).columns.tolist()
    #drop geometry column from numeric columns
    if 'geometry' in numeric_cols:
        numeric_cols.remove('geometry')
    if watershed_col in numeric_cols:
        numeric_cols.remove(watershed_col)
    gdf[numeric_cols] = gdf[numeric_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
    
    # Ensure all columns except the watershed column are of type string
    non_watershed_cols = gdf.columns.drop(watershed_col)
    gdf[non_watershed_cols] = gdf[non_watershed_cols].astype(str)
    
    return gdf

In [54]:
import os
import attribute_params as params
import warnings
warnings.filterwarnings('ignore', category=FutureWarning, module='geopandas')


channel_poly_dir = r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Watershed Stats\Channels\Channel Polygons\Segmented"
centerline_dir = r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Watershed Stats\Channels\Centerlines"
output_dir = r"Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Watershed Stats\Channels\Channel Polygons\Channel Stats"
field_list = ['Area', 'Flow Accumulation Max', 'Slope Over Area']
watersheds = ['LM2', 'LPM', 'MM', 'MPM', 'UM1', 'UM2']
#watersheds = ['LM2']

for watershed in watersheds:
    print(f"Processing {watershed}\n")
    for file in os.listdir(channel_poly_dir):
        if watershed in file:
            channel_poly_path = os.path.join(channel_poly_dir, file)
            break
    for file in os.listdir(centerline_dir):
        if watershed in file:
            centerline_path = os.path.join(centerline_dir, file)
            break
    output_path = os.path.join(output_dir, f'{watershed}_channel_stats.gpkg')
    
    elevation_params = params.build_params('Elevation', ['mean'], watershed)
    erosion_params = params.build_params('Erosion', ['mean', 'sum', 'count'], watershed)
    deposition_params = params.build_params('Deposition', ['mean', 'sum', 'count'], watershed)
    sfm_erosion_params = params.build_params('SfM Erosion', ['mean', 'sum', 'count'], watershed)
    sfm_deposition_params = params.build_params('SfM Deposition', ['mean', 'sum', 'count'], watershed)
    flow_accum_params = params.build_params('Flow Accumulation', ['max'], watershed)
    slope_params = params.build_params('Slope', ['mean'], watershed)
    print("Calculating raster stats")
    gdf = gpd.read_file(channel_poly_path)
    #Add a field for watershed
    gdf['Watershed'] = watershed
    gdf = aggregate_raster_stats(gdf, output_path, elevation_params)
    gdf = add_area_field(output_path, output_path)
    
    gdf = aggregate_raster_stats(gdf, output_path, erosion_params)
    gdf = aggregate_raster_stats(gdf, output_path, deposition_params)
    gdf = calc_net_change(output_path)
    
    try:
        gdf = aggregate_raster_stats(gdf, output_path, sfm_erosion_params)
        gdf = aggregate_raster_stats(gdf, output_path, sfm_deposition_params)
        gdf = calc_net_sfm_change(output_path)
    except:
        print(f"Skipping SfM stats for {watershed}")
    
    gdf = aggregate_raster_stats(gdf, output_path, slope_params)
    gdf = aggregate_raster_stats(gdf, output_path, flow_accum_params)
    
    print("Calculating area, width, slope")
    
    calculate_width(output_path, centerline_path, output_path)
    calculate_forward_slope_difference(output_path, output_path)
    calculate_backward_slope_difference(output_path, output_path)
    calculate_central_slope_difference(output_path, output_path)
    print("Calculating curvature")
    add_curvature_field(output_path, centerline_path, output_path)
    #add_slope_over_width(output_path)
    #add_change_in_slope_over_width(output_path)
    add_change_in_slope_over_area(output_path)
    add_slope_over_area(output_path)

    for field in field_list:
        gdf = calculate_central_difference(output_path, output_path, field)
    


Processing LM2

Calculating raster stats
Calculating area, width, slope
Calculating curvature
Polygon 136 does not intersect with the centerline
1
['Area']
1
['Flow Accumulation Max']
1
['Slope Over Area']
Processing LPM

Calculating raster stats
Calculating area, width, slope
Calculating curvature
1
['Area']
1
['Flow Accumulation Max']
1
['Slope Over Area']
Processing MM

Calculating raster stats
Calculating area, width, slope
Calculating curvature
Polygon 173 does not intersect with the centerline
Polygon 174 does not intersect with the centerline
Polygon 175 does not intersect with the centerline
Polygon 176 does not intersect with the centerline
Polygon 177 does not intersect with the centerline
Polygon 178 does not intersect with the centerline
Polygon 179 does not intersect with the centerline
Polygon 180 does not intersect with the centerline
Polygon 181 does not intersect with the centerline
Polygon 182 does not intersect with the centerline
Polygon 183 does not intersect with 

C:\Users\alextd\AppData\Local\Temp\3\ipykernel_27980\355849106.py:60: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  curvature = max(calculate_curvature(line) for line in intersection if isinstance(line, LineString))


1
['Area']
1
['Flow Accumulation Max']
1
['Slope Over Area']
Processing UM1

Calculating raster stats
Skipping SfM stats for UM1
Calculating area, width, slope
Calculating curvature
Polygon 0 does not intersect with the centerline
Polygon 1 does not intersect with the centerline
Polygon 2 does not intersect with the centerline
Polygon 3 does not intersect with the centerline
Polygon 4 does not intersect with the centerline
Polygon 5 does not intersect with the centerline
Polygon 6 does not intersect with the centerline
Polygon 7 does not intersect with the centerline
Polygon 8 does not intersect with the centerline
Polygon 9 does not intersect with the centerline
Polygon 10 does not intersect with the centerline
Polygon 11 does not intersect with the centerline
Polygon 12 does not intersect with the centerline
Polygon 13 does not intersect with the centerline
Polygon 14 does not intersect with the centerline
Polygon 15 does not intersect with the centerline
Polygon 16 does not intersec

In [57]:

from vector_utils import combine_attributes_and_save_to_csv
combine_attributes_and_save_to_csv(output_dir)




CSV files have been concatenated and saved to: Y:\ATD\GIS\East_Troublesome\Watershed Statistical Analysis\Watershed Stats\Channels\Channel Polygons\Channel Stats\csv\combined_attributes.csv
